In [1]:
# 루트 탐색 & 이동 셀 (노트북 첫 셀 -- 재실행해도 안전)
from pathlib import Path
import os

p = Path.cwd()
while p != p.parent and not (p / "data" / "raw").exists():
    p = p.parent

if (p / "data" / "raw").exists():
    os.chdir(p)
    print("✔️ 루트 고정:", Path.cwd())
    print("▶ data/raw:", [f.name for f in (p/"data"/"raw").iterdir()])
else:
    raise RuntimeError("❌ 프로젝트 루트를 찾지 못했습니다 -- VSCode에서 루트 폴더를 열었는지 확인하세요.")


✔️ 루트 고정: /Users/bichae/active-zones-report
▶ data/raw: ['서울시 상권분석서비스(추정매출-상권).csv', '서울시 상권분석서비스(길단위인구-상권).csv', '서울시 상권분석서비스(상주인구-상권).csv']


In [2]:
import pandas as pd
df = pd.read_csv("data/processed/merged_annual_2024.csv", encoding="utf-8-sig")
df.head()


,상권_코드,resident_pop_2024,floating_pop_2024,estimated_sales_2024
0,3001491,105840,52278828,451309752219
1,3001492,21240,173404731,2323168283547
2,3001493,54746,84336760,592441401861
3,3001494,38850,217027839,1233135489141
4,3001495,202576,96379422,1797325989068


In [3]:
# ── 셀 0: 언제 어디서 실행해도 루트를 찾아줌 ─────────────────────────
from pathlib import Path
import os

p = Path.cwd()
while p != p.parent and not (p / "data" / "raw").exists():
    p = p.parent

if (p / "data" / "raw").exists():
    os.chdir(p)
    print("✔️ 프로젝트 루트로 이동:", p)
    print("▶ data/raw:", [f.name for f in (p/'data'/'raw').iterdir()])
else:
    raise FileNotFoundError("data/raw 폴더를 찾지 못했습니다.")


✔️ 프로젝트 루트로 이동: /Users/bichae/active-zones-report
▶ data/raw: ['서울시 상권분석서비스(추정매출-상권).csv', '서울시 상권분석서비스(길단위인구-상권).csv', '서울시 상권분석서비스(상주인구-상권).csv']


In [4]:
import os

print("▶ data/raw 폴더 내용:")
for fname in os.listdir("data/raw"):
    print("  -", fname)


▶ data/raw 폴더 내용:
  - 서울시 상권분석서비스(추정매출-상권).csv
  - 서울시 상권분석서비스(길단위인구-상권).csv
  - 서울시 상권분석서비스(상주인구-상권).csv


In [5]:
import pandas as pd

# 1) 파일 경로
resident_fp = "data/raw/서울시 상권분석서비스(상주인구-상권).csv"
floating_fp = "data/raw/서울시 상권분석서비스(길단위인구-상권).csv"
sales_fp    = "data/raw/서울시 상권분석서비스(추정매출-상권).csv"

# 2) CSV 로드
df_res   = pd.read_csv(resident_fp, encoding="cp949")
df_flo   = pd.read_csv(floating_fp, encoding="cp949")
df_sales = pd.read_csv(sales_fp,    encoding="cp949")

# 3) 연간 합계 집계
res_yr = (
    df_res
    .groupby("상권_코드")["총_상주인구_수"]
    .sum()
    .reset_index()
    .rename(columns={"총_상주인구_수":"resident_pop_2024"})
)

flo_yr = (
    df_flo
    .groupby("상권_코드")["총_유동인구_수"]
    .sum()
    .reset_index()
    .rename(columns={"총_유동인구_수":"floating_pop_2024"})
)

sales_yr = (
    df_sales
    .groupby("상권_코드")["당월_매출_금액"]
    .sum()
    .reset_index()
    .rename(columns={"당월_매출_금액":"estimated_sales_2024"})
)

# 4) 병합 & 저장
df_annual = res_yr.merge(flo_yr, on="상권_코드") \
                .merge(sales_yr, on="상권_코드")

df_annual.to_csv(
    "data/processed/merged_annual_2024.csv",
    index=False,
    encoding="utf-8-sig"
)

print("✅ 연간 집계 완료, 결과 크기:", df_annual.shape)
df_annual.head()


✅ 연간 집계 완료, 결과 크기: (1571, 4)


,상권_코드,resident_pop_2024,floating_pop_2024,estimated_sales_2024
0,3001491,105840,52278828,451309752219
1,3001492,21240,173404731,2323168283547
2,3001493,54746,84336760,592441401861
3,3001494,38850,217027839,1233135489141
4,3001495,202576,96379422,1797325989068
